In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
import csv
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
shadow_model = load_model('model\\shadow2_T_1.h5')

In [4]:
start_index = 2000
shadow_test_x = x_train[start_index:start_index + 2000]
for i in range(1,6):
    left = i * 10000 + start_index
    shadow_test_x = np.concatenate((shadow_test_x,x_train[left:left + 2000]), axis=0)
shadow_test_x = np.concatenate((shadow_test_x,x_test[start_index:start_index + 2000]), axis=0)

In [5]:
shadow_test_y = y_train[start_index:start_index + 2000]
for i in range(1,6):
    left = i * 10000 + start_index
    shadow_test_y = np.concatenate((shadow_test_y,y_train[left:left + 2000]), axis=0)
shadow_test_y = np.concatenate((shadow_test_y,y_test[start_index:start_index + 2000]), axis=0)

In [6]:
print(shadow_test_x.shape)
print(shadow_test_y.shape)

(14000, 28, 28, 1)
(14000, 10)


In [7]:
shadow_test_x = shadow_test_x.reshape(-1, 28 * 28)
print(shadow_test_x.shape)

(14000, 784)


In [8]:
score = shadow_model.evaluate(shadow_test_x, shadow_test_y, verbose=1)

14000/14000 [==============================] - 1s 101us/step


In [9]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.12299082535606742
Test accuracy: 0.9658571428571429


In [10]:
shadow_predict = shadow_model.predict(shadow_test_x)

In [11]:
print(shadow_predict.shape)

(14000, 10)


In [12]:
shadow_label = np.zeros((start_index + 2000,1))
shadow_label = np.concatenate((shadow_label,np.ones((2000,1))), axis=0)
shadow_label = np.concatenate((shadow_label,np.zeros((14000 - start_index - 4000,1))), axis=0)

In [13]:
print(shadow_label.shape)

(14000, 1)


In [14]:
test_result_shadow = np.concatenate((shadow_predict,shadow_label),axis=1)

In [15]:
print(test_result_shadow.shape)

(14000, 11)


In [16]:
np.savetxt('model\\test_shadow2.csv',test_result_shadow,delimiter=',')